In [2]:
import pandas as pd
import fastf1 
import numpy as np

# Limiting the timeframe of interest
In order to limit the timeframe of interest for the reserch we will only focus on the past 5 completed seasons of F1, namely 2018-2024. 
To do so we first need to collect a complete calendar of all the events ran in this timeframe


In [3]:
import fastf1.events

# Collecting the calendar
complete_calendar = fastf1.events.get_event_schedule(2018)

for year in range(2019, 2025):
    complete_calendar = pd.concat([complete_calendar, fastf1.events.get_event_schedule(year)])

complete_calendar.head()

req         WARNING 	DEFAULT CACHE ENABLED! (24.0 KB) C:\Users\vnvtr\AppData\Local\Temp\fastf1


,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,1,Australia,Melbourne,FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX,2018-03-25,Australian Grand Prix,conventional,Practice 1,2018-03-23 12:00:00+11:00,2018-03-23 01:00:00,...,Practice 3,2018-03-24 14:00:00+11:00,2018-03-24 03:00:00,Qualifying,2018-03-24 17:00:00+11:00,2018-03-24 06:00:00,Race,2018-03-25 16:10:00+11:00,2018-03-25 05:10:00,True
1,2,Bahrain,Sakhir,FORMULA 1 2018 GULF AIR BAHRAIN GRAND PRIX,2018-04-08,Bahrain Grand Prix,conventional,Practice 1,2018-04-06 14:00:00+03:00,2018-04-06 11:00:00,...,Practice 3,2018-04-07 15:00:00+03:00,2018-04-07 12:00:00,Qualifying,2018-04-07 18:00:00+03:00,2018-04-07 15:00:00,Race,2018-04-08 18:10:00+03:00,2018-04-08 15:10:00,True
2,3,China,Shanghai,FORMULA 1 2018 HEINEKEN CHINESE GRAND PRIX,2018-04-15,Chinese Grand Prix,conventional,Practice 1,2018-04-13 10:00:00+08:00,2018-04-13 02:00:00,...,Practice 3,2018-04-14 11:00:00+08:00,2018-04-14 03:00:00,Qualifying,2018-04-14 14:00:00+08:00,2018-04-14 06:00:00,Race,2018-04-15 14:10:00+08:00,2018-04-15 06:10:00,True
3,4,Azerbaijan,Baku,FORMULA 1 2018 AZERBAIJAN GRAND PRIX,2018-04-29,Azerbaijan Grand Prix,conventional,Practice 1,2018-04-27 13:00:00+04:00,2018-04-27 09:00:00,...,Practice 3,2018-04-28 14:00:00+04:00,2018-04-28 10:00:00,Qualifying,2018-04-28 17:00:00+04:00,2018-04-28 13:00:00,Race,2018-04-29 16:10:00+04:00,2018-04-29 12:10:00,True
4,5,Spain,Barcelona,FORMULA 1 GRAN PREMIO DE ESPAÑA EMIRATES 2018,2018-05-13,Spanish Grand Prix,conventional,Practice 1,2018-05-11 11:00:00+02:00,2018-05-11 09:00:00,...,Practice 3,2018-05-12 12:00:00+02:00,2018-05-12 10:00:00,Qualifying,2018-05-12 15:00:00+02:00,2018-05-12 13:00:00,Race,2018-05-13 15:10:00+02:00,2018-05-13 13:10:00,True


In [4]:
complete_calendar['EventFormat'].value_counts()

EventFormat
conventional         131
testing                7
sprint                 6
sprint_shootout        6
sprint_qualifying      6
Name: count, dtype: int64

Now lets remove testing sessions from the calendar since information may be incomplete and any insights may be biased because of 'bagging' practices. 


In [5]:
complete_calendar = complete_calendar[complete_calendar['EventFormat'].str.contains('testing') == False]
complete_calendar['EventFormat'].value_counts()

EventFormat
conventional         131
sprint                 6
sprint_shootout        6
sprint_qualifying      6
Name: count, dtype: int64

Lets inspect the 3 different types of sprint events and verify that they are actually different event formats. 

In [6]:
complete_calendar[complete_calendar['EventFormat'].str.contains('sprint')]

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
10,10,Great Britain,Silverstone,FORMULA 1 PIRELLI BRITISH GRAND PRIX 2021,2021-07-18,British Grand Prix,sprint,Practice 1,2021-07-16 14:30:00+01:00,2021-07-16 13:30:00,...,Practice 2,2021-07-17 12:00:00+01:00,2021-07-17 11:00:00,Sprint,2021-07-17 16:30:00+01:00,2021-07-17 15:30:00,Race,2021-07-18 15:00:00+01:00,2021-07-18 14:00:00,True
14,14,Italy,Monza,FORMULA 1 HEINEKEN GRAN PREMIO D’ITALIA 2021,2021-09-12,Italian Grand Prix,sprint,Practice 1,2021-09-10 14:30:00+02:00,2021-09-10 12:30:00,...,Practice 2,2021-09-11 12:00:00+02:00,2021-09-11 10:00:00,Sprint,2021-09-11 16:30:00+02:00,2021-09-11 14:30:00,Race,2021-09-12 15:00:00+02:00,2021-09-12 13:00:00,True
19,19,Brazil,São Paulo,FORMULA 1 HEINEKEN GRANDE PRÊMIO DE SÃO PAULO ...,2021-11-14,São Paulo Grand Prix,sprint,Practice 1,2021-11-12 12:30:00-03:00,2021-11-12 15:30:00,...,Practice 2,2021-11-13 12:00:00-03:00,2021-11-13 15:00:00,Sprint,2021-11-13 16:30:00-03:00,2021-11-13 19:30:00,Race,2021-11-14 14:00:00-03:00,2021-11-14 17:00:00,True
5,4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...,2022-04-24,Emilia Romagna Grand Prix,sprint,Practice 1,2022-04-22 13:30:00+02:00,2022-04-22 11:30:00,...,Practice 2,2022-04-23 12:30:00+02:00,2022-04-23 10:30:00,Sprint,2022-04-23 16:30:00+02:00,2022-04-23 14:30:00,Race,2022-04-24 15:00:00+02:00,2022-04-24 13:00:00,True
12,11,Austria,Spielberg,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2022,2022-07-10,Austrian Grand Prix,sprint,Practice 1,2022-07-08 13:30:00+02:00,2022-07-08 11:30:00,...,Practice 2,2022-07-09 12:30:00+02:00,2022-07-09 10:30:00,Sprint,2022-07-09 16:30:00+02:00,2022-07-09 14:30:00,Race,2022-07-10 15:00:00+02:00,2022-07-10 13:00:00,True
22,21,Brazil,São Paulo,FORMULA 1 HEINEKEN GRANDE PRÊMIO DE SÃO PAULO ...,2022-11-13,São Paulo Grand Prix,sprint,Practice 1,2022-11-11 12:30:00-03:00,2022-11-11 15:30:00,...,Practice 2,2022-11-12 12:30:00-03:00,2022-11-12 15:30:00,Sprint,2022-11-12 16:30:00-03:00,2022-11-12 19:30:00,Race,2022-11-13 15:00:00-03:00,2022-11-13 18:00:00,True
4,4,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023-04-30,Azerbaijan Grand Prix,sprint_shootout,Practice 1,2023-04-28 13:30:00+04:00,2023-04-28 09:30:00,...,Sprint Shootout,2023-04-29 12:30:00+04:00,2023-04-29 08:30:00,Sprint,2023-04-29 17:30:00+04:00,2023-04-29 13:30:00,Race,2023-04-30 15:00:00+04:00,2023-04-30 11:00:00,True
9,9,Austria,Spielberg,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2023,2023-07-02,Austrian Grand Prix,sprint_shootout,Practice 1,2023-06-30 13:30:00+02:00,2023-06-30 11:30:00,...,Sprint Shootout,2023-07-01 12:00:00+02:00,2023-07-01 10:00:00,Sprint,2023-07-01 16:30:00+02:00,2023-07-01 14:30:00,Race,2023-07-02 15:00:00+02:00,2023-07-02 13:00:00,True
12,12,Belgium,Spa-Francorchamps,FORMULA 1 MSC CRUISES BELGIAN GRAND PRIX 2023,2023-07-30,Belgian Grand Prix,sprint_shootout,Practice 1,2023-07-28 13:30:00+02:00,2023-07-28 11:30:00,...,Sprint Shootout,2023-07-29 12:00:00+02:00,2023-07-29 10:00:00,Sprint,2023-07-29 17:05:00+02:00,2023-07-29 15:05:00,Race,2023-07-30 15:00:00+02:00,2023-07-30 13:00:00,True
17,17,Qatar,Lusail,FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2023,2023-10-08,Qatar Grand Prix,sprint_shootout,Practice 1,2023-10-06 16:30:00+03:00,2023-10-06 13:30:00,...,Sprint Shootout,2023-10-07 16:20:00+03:00,2023-10-07 13:20:00,Sprint,2023-10-07 20:30:00+03:00,2023-10-07 17:30:00,Race,2023-10-08 20:00:00+03:00,2023-10-08 17:00:00,True


Since the 3 different types of sprint events actually correspond to the same event format we are going to change the value of the variable 'EventFormat' to just 'spint'.

In [7]:
complete_calendar.loc[:, 'EventFormat'] = complete_calendar['EventFormat'].str.replace('sprint_qualifying', 'sprint')
complete_calendar.loc[:, 'EventFormat'] = complete_calendar['EventFormat'].str.replace('sprint_shootout', 'sprint')
complete_calendar['EventFormat'].value_counts()

EventFormat
conventional    131
sprint           18
Name: count, dtype: int64

In [8]:
complete_calendar.to_csv('complete_calendar.csv', index=False)

# Extracting teams and drivers
We can now extract drivers and team names by calling .get_drivers() for each event. To save on network usage we will locally cache the fastf1 data collected. 

In [9]:
import fastf1.logger as logger
import logging


fastf1.logger.LoggingManager().set_level(logging.CRITICAL)

In [29]:
import pandas as pd
import fastf1
from tqdm.auto import tqdm # Import tqdm

# Assuming 'complete_calendar' DataFrame is loaded and preprocessed

fastf1.Cache.enable_cache('./cache')
fastf1.Cache.offline_mode(enabled=True)

# Extracting results from each race
results = []
failed_sessions = []

# Get the total number of events to process for the progress bar
total_events = len(complete_calendar) 

# Use tqdm by wrapping the iteration over the DataFrame rows
# Use .iterrows() to get both index and row data
print(f"Processing {total_events} events...")
for index, row in tqdm(complete_calendar.iterrows(), total=total_events, desc="Loading Race Results"):
    year = row['EventDate'].year # Extract year from the row
    event = row['EventName']     # Extract event name from the row
    race_identifier = f"{year} - {event}" # Create identifier

    try:
        session = fastf1.get_session(year, event, 'R')
        session.load() 
        
        event_results = session.results
        
        if event_results is not None and not event_results.empty:
            # Use .copy() to avoid potential SettingWithCopyWarning
            event_results = event_results.copy() 
            event_results['Event'] = event
            event_results['Year'] = year
            results.append(event_results)
        else:
            print(f"Warning: No results data found for {race_identifier}")
            failed_sessions.append(race_identifier + " (No results)")

    except Exception as e:
        print(f"Error loading session data for {race_identifier}: {e}")
        failed_sessions.append(f"{race_identifier} (Error: {e})")

Processing 149 events...


Loading Race Results:   0%|          | 0/149 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [40]:
complete_results.to_csv('complete_results.csv', index=False)

# Checkpoint: reading complete results

In [18]:
complete_results = pd.read_csv('complete_results.csv')
complete_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DriverNumber        2979 non-null   int64  
 1   BroadcastName       2976 non-null   object 
 2   Abbreviation        2979 non-null   object 
 3   DriverId            2899 non-null   object 
 4   TeamName            2979 non-null   object 
 5   TeamColor           2976 non-null   object 
 6   TeamId              2899 non-null   object 
 7   FirstName           2979 non-null   object 
 8   LastName            2979 non-null   object 
 9   FullName            2979 non-null   object 
 10  HeadshotUrl         2354 non-null   object 
 11  CountryCode         1357 non-null   object 
 12  Position            2896 non-null   float64
 13  ClassifiedPosition  2899 non-null   object 
 14  GridPosition        2896 non-null   float64
 15  Q1                  0 non-null      float64
 16  Q2    

In [27]:
complete_results[complete_results['DriverId'].isna()]['Event'].value_counts()

Event
Hungarian Grand Prix    20
Belgian Grand Prix      20
Dutch Grand Prix        20
Italian Grand Prix      20
Name: count, dtype: int64

There are multiple missing values for all drivers in 4 2024 races. To be investigated ... 

In [13]:
complete_results = pd.concat(results)
complete_results.info()

<class 'fastf1.core.SessionResults'>
Index: 2979 entries, 5 to 11
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   DriverNumber        2979 non-null   object         
 1   BroadcastName       2979 non-null   object         
 2   Abbreviation        2979 non-null   object         
 3   DriverId            2979 non-null   object         
 4   TeamName            2979 non-null   object         
 5   TeamColor           2979 non-null   object         
 6   TeamId              2979 non-null   object         
 7   FirstName           2979 non-null   object         
 8   LastName            2979 non-null   object         
 9   FullName            2979 non-null   object         
 10  HeadshotUrl         2979 non-null   object         
 11  CountryCode         2979 non-null   object         
 12  Position            2896 non-null   float64        
 13  ClassifiedPosition  2979 non-null   objec

In [18]:
complete_results['Status'].value_counts()

Status
Finished            1633
+1 Lap               536
Lapped               182
+2 Laps              101
Retired               89
                      80
Collision             77
Accident              35
Engine                31
Collision damage      29
Brakes                20
Power Unit            16
Gearbox               15
+3 Laps               12
Suspension            10
Disqualified           9
Hydraulics             8
Power loss             8
Wheel                  7
Did not start          6
Puncture               6
Withdrew               5
Spun off               4
Water pressure         4
Overheating            4
Oil leak               4
Fuel pressure          4
Exhaust                4
Mechanical             3
Electronics            3
Turbo                  3
Undertray              3
Water leak             2
Illness                2
Transmission           2
Damage                 2
Electrical             2
+5 Laps                1
Debris                 1
Steering          

In [26]:
complete_results['DNF'] = complete_results.apply(lambda row: False if row['Status'] in ['Finished', '+1 Lap',
                                                                                            '+2 Laps', '+3 Laps', '+4 Laps', '+5 Laps', '+6 Laps',
                                                                                            'Lapped'] else True, axis=1)

In [45]:
# Group by DriverId and aggregate the required information
driver_summary = complete_results.groupby('FullName').agg(
    # Calculate the sum of 'Points' for each driver
    TotalPoints=('Points', 'sum'),

    # Count the total number of race result entries for each driver
    NumberOfStarts=('DriverId', 'size'),
    
    # Number of races with a DNF
    NumberOfDNF=('DNF', lambda x: x.sum()),
    
    # DNF percentage
    DNF_Percentage=('DNF', lambda x: x.sum() / len(x)),

    # Collect a unique list of 'TeamId's each driver has raced for
    TeamIds=('TeamId', lambda x: list(x.unique())),
    
    # Lowest grid position
    LowestFinishPosition=('Position', 'min'),

    # Highest grid position
    HighestFinishPosition=('Position', 'max'),
    
    # Average grid position
    AverageFinishPosition=('Position', 'mean'),    
    
    # Number of wins
    NumberOfWins=('ClassifiedPosition', lambda x: (x == '1').sum()),
    
    # Number of podiums
    #NumberOfPodiums=('ClassifiedPosition', lambda x: (x.astype(int) <= 3).sum()),
    
    # Number of poles
    #NumberOfPoles=('ClassifiedPosition', lambda x: (x.astype(int) == 1).sum()),
    
    
).reset_index() # Convert the grouped 'DriverId' back into a column

# Optional: Sort the summary table by TotalPoints in descending order
driver_summary = driver_summary.sort_values(by='TotalPoints', ascending=False)

# Display the resulting summary table
print("Driver Summary Table:")
print(driver_summary)

# Display info about the summary table structure
print("\nSummary Table Info:")
driver_summary.info()

Driver Summary Table:
              FullName  TotalPoints  NumberOfStarts  NumberOfDNF  \
22      Max Verstappen       2443.5             149           21   
18      Lewis Hamilton       2156.5             148           12   
4      Charles Leclerc       1296.0             149           28   
36        Sergio Perez       1093.0             147           22   
38     Valtteri Bottas       1072.0             149           24   
3         Carlos Sainz       1047.5             148           23   
17        Lando Norris        880.0             128           15   
34    Sebastian Vettel        673.0             101           14   
10      George Russell        647.0             128           21   
5     Daniel Ricciardo        503.0             128           25   
8      Fernando Alonso        475.0             111           23   
30        Pierre Gasly        424.0             149           26   
7         Esteban Ocon        351.0             127           27   
15      Kimi Räikkönen    

In [46]:
driver_summary.to_csv('driver_summary.csv', index=False)